# Case study: Estimating the orbit of _Beta Pictoris b_

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Fill in this cell with your personal details:
# - Name: xxx
# - Student ID: xxx
# - Email: xxx

Deadline: December 19th 2025.

Instructions:
- Submit your completed notebook on Gradescope.
- All cells must be executable and their outputs should not be erased before submission.
- This case study must be carried out alone. You are not allowed to discuss or collaborate with other students.
- In January, you will have to explain and defend your solution during a 30-minute oral exam.
- This case study (including its defense in January) will account for 50% of the final grade.

Comments:
- Solve the case study below by following the Bayesian workflow as best as possible.
- Follow the data visualization principles to make your plots effective and readable.
- Make the best use of the Python scientific ecosystem.
- Add comments to explain your reasoning, choices, and interpretations.
- You can add additional cells as needed.

# Introduction

Beta Pictoris is a young star located approximately 63 light-years from Earth in the constellation Pictor. It has been a target of intense astronomical study since the 1980s, when astronomers discovered a large debris disk surrounding the star. In 2008, astronomers using direct imaging techniques discovered a giant exoplanet orbiting Beta Pictoris, named _Beta Pictoris b_.

Since its discovery, _Beta Pictoris b_ has been observed multiple times using various telescopes and instruments tracking its position relative to the host star. These observations have provided valuable data on the planet's orbit, which is crucial for understanding its formation and evolution, as well as the dynamics of the surrounding debris disk.

Your goal in this case study is to exercise your Bayesian data analysis skills to estimate the orbit of _Beta Pictoris b_ using the available astrometric data.

<div align="center">
<img src="./betapicb.gif" alt="Beta Pictoris system" width="500">
<p><i>Credit: Gemini Planet Imager, VLT/NACO, and VLT/SPHERE.</i></p>
</div>

# Data Exploration

<div class="alert alert-success">
    
**Q1**. Load the astrometric data from `beta_pic_b_sep_pa.csv` into a Pandas DataFrame.
    
</div>

<div class="alert alert-success">
    
**Q2**. Explore the dataset and report relevant insights.
    
</div>

# Build: Keplerian orbits

We recommend watching the following [video](https://www.youtube.com/watch?v=yn_Nto4Bd48) for a visual explanation of Keplerian orbits and their parameters but provide a summary below.

The motion of Beta Pictoris b around its star follows Kepler's laws of planetary motion. A full three-dimensional orbit is defined by eight orbital parameters that describes its shape and orientation:

- **Semi-major axis** ($a$): half of the long axis of the ellipse [AU]<sup>(1)</sup>. The short axis (semi-minor), is related to the semi-major axis and eccentricity by $b = a \sqrt{1 - e^2}$.
- **Eccentricity** ($e$): how elongated the ellipse is (e $\in [0, 1]$).
- **Inclination** ($i$): tilt of the orbital plane w.r.t reference plane which is the plane perpendicular to the line of sight from Earth to the star (in degrees or radians).
- **Argument of periapsis** ($\omega$): the periapsis is the point where the planet is closest to the star. All orbital angles are measured relative to that point. The argument of periapsis defines the angle from the ascending node (the point where the planet crosses the reference plane going "up") to the periapsis (in radians).
- **Position angle of ascending node** ($\Omega$): the rotation of the orbital plane around the line of sight (in radians). It orients the orbital plane on the sky.
- **Time of periapsis** ($\tau$): what fraction of the orbital period ago was the last periapsis passage (in fraction of period, between 0 and 1).
- **Parallax** ($\Pi$): the angular shift of the star due to Earth's orbit, which tells us the distance to the star [mas]<sup>(2)</sup>.
- **Total mass** ($M_T$): the combined mass of the star and planet (in solar masses [$M_\odot$]).

<div align="center">
<img src="./kepler_arguments.png" alt="Kepler parameters illustration" width="500">
<p><i>Kepler parameters. Credit: Sun et al. (2022), arxiv:2201.08506</i></p>
</div>

From these parameters, we first derive the orbital period $P$ (in years) using Kepler's third law:
$$\begin{align*}
(\frac{P}{1 \text{ year}})^2 &= (\frac{a}{1 \text{ au}})^3 \; \left(\frac{M_T}{ 1 \; M_\odot}\right)^{-1}.
\end{align*}$$

The angular offsets $\Delta \mathrm{RA}$ and $\Delta \mathrm{Dec}$ at each observation epoch $t$ of the planet relative to the host star on the reference plane can then be computed using Kepler's equations through intermediate angular quantities called anomalies. The term "anomaly" is a historical term meaning angles measured from reference directions. We have three types of anomalies:

- Mean anomaly
$$ \begin{align*}
M = 2\pi \Big(\frac{t - t_{\text{ref}}}{P} - \tau\Big),
\end{align*}
$$
which increases uniformly with time. It tracks orbital phase as if the planet moved at constant speed. $t_{\text{ref}}$ is a reference epoch for $\tau$ (here $t_{\text{ref}} = 50000 [MJD]$ <sup>(3)</sup>, which corresponds to 1995-10-10).
- Eccentric anomaly ($E$): an auxiliary angle defined implicitly by Kepler’s equation ($M = E - e\sin E$). $E$ cannot be computed analytically (i.e., no closed form algebraic solution exists), so we will need to solve it numerically (e.g., using the Newton-Raphson method).
- True anomaly ($\nu$): the geometric angle locating the planet along the orbit, related to $E$ by
$$\begin{align*}
\nu &= 2 \arctan \left( \sqrt{\frac{1+e}{1-e}} \tan \frac{E}{2} \right).
\end{align*}$$

$M$, $E$, and $\nu$ are not parameters but computed intermediate angles.

From these, we can finally derive the angular offsets of the planet relative to the star. Projecting the orbit into the observer's frame, we have:

$$ \begin{align*}
\Delta \mathrm{RA} &= \Pi a (1 - e \cos E) \left[\cos^2 \frac{i}{2} \; \sin(\nu + \omega + \Omega) - \sin^2 \frac{i}{2} \; \sin(\nu + \omega - \Omega)\right], \\
\Delta \mathrm{Dec} &= \Pi a (1 - e \cos E) \left[\cos^2 \frac{i}{2} \; \cos(\nu + \omega + \Omega) + \sin^2 \frac{i}{2} \; \cos(\nu + \omega - \Omega)\right].
\end{align*} $$

$\Delta\mathrm{RA}$ and $\Delta\mathrm{Dec}$ are angular displacements, not physical distances:
one measures how far east–west, the other north–south, the planet appears from the star.
The $\Delta$ symbol indicates that these are relative to the star’s position, not absolute celestial coordinates.
They depend only on orbital geometry and orientation, not on detector properties except for calibration.

However, telescopes do not directly measure Cartesian angular offsets $(\Delta\mathrm{RA},\Delta\mathrm{Dec})$. Instead, they measure two polar quantities:

- Separation (sep): the apparent angular distance between the star and planet,
- Position angle (PA): the direction of that offset, measured east of north.

These are related to the Cartesian offsets by the following equations:
$$ \begin{align*}
\Delta \mathrm{RA} &= \text{sep} \times \sin(\text{PA}), \\
\Delta \mathrm{Dec} &= \text{sep} \times \cos(\text{PA}).
\end{align*} $$

Converting between the two allows a direct comparison between the theoretical orbit and the observed astrometric data.

---

(1): Astronomical Unit (AU) is the average distance between the Earth and the Sun, approximately 149.6 million kilometers.

(2): Milliarcseconds (mas), where 1 arcsecond = 1/3600 degrees, and 1 mas = 1/1000 arcseconds.

(3): Modified Julian Date (MJD) is a continuous count of days since midnight on November 17, 1858. It is commonly used in astronomy to simplify date calculations.


<div class="alert alert-success">
    
**Q3**. Implement `solve_kepler` and `compute_orbit` functions to compute the predicted $\Delta \mathrm{RA}$ and $\Delta \mathrm{Dec}$ at given times (epochs). 
    
</div>

In [ ]:
def solve_kepler(M, ecc):
    """
    Solve Kepler's equation: M = E - ecc*sin(E) for eccentric anomaly E.
    
    Parameters:
    -----------
        - M: mean anomaly in radians
        - ecc: eccentricity (0 <= ecc < 1)

    Returns:
    --------
        - E: eccentric anomaly in radians
    """
    pass # Your implementation here


def compute_orbit(epochs, sma, ecc, inc, aop, pan, tau, plx, mtot, t_ref=50000.0):
    """
    Calculate predicted separation and position angle from Keplerian orbital elements
    
    Parameters:
    -----------
        - epochs: array of observation epochs (t1, t2, ..., tn) in [MJD]
        - sma: semi-major axis [AU]
        - ecc: eccentricity [0, 1]
        - inc: inclination [radians]
        - aop: argument of periastron (omega) [radians]
        - pan: position angle of nodes (Omega) [radians]
        - tau: epoch of periastron passage (fraction of period)
        - plx: parallax [mas]
        - mtot: total system mass [solar masses]
        - t_ref: reference epoch [MJD]

    Returns:
    --------
        - sep: predicted separation [mas]
        - pa: predicted position angle [°]
    """
    pass # Your implementation here

<div class="alert alert-success">
    
**Q4**. Implement a Bayesian model with:

- **Prior** $p(\theta)$ on the orbital parameters $\theta = (a, e, i, \omega, \Omega, \tau, \Pi, M_{tot})$. Hint: The parallax $\Pi$ and total mass $M_{tot}$ can be constrained using existing measurements of the host star Beta Pictoris.
    - Define appropriate prior distributions for each orbital parameter.
    - Guide your choices with prior predictive checks and adjust as needed.

- **Likelihood** $p((\text{sep}_{t_1}, \text{pa}_{t_1}), ..., (\text{sep}_{t_n}, \text{pa}_{t_n})|\theta)$ that models the measured separations [mas] and position angles [degrees] given the orbital parameters $\theta$. Epochs $t_1, \ldots, t_n$ correspond to the times of the observations. Gaussian errors are assumed for both measurements, with standard deviations given by `sep_err` and `pa_err` for each epoch.

Define functions both for evaluating the log densities and for sampling from the prior and likelihood at given epochs and standard deviations.
    
</div>

<div class="alert alert-success">
    
**Q5**. Draw the probabilistic graphical model. What are the observed variables? What are the latent variables? What are the hyperparameters?
    
</div>

# Compute

You are given the dataset `beta_pic_b_sep_pa.csv`, which contains astrometric measurements of Beta Pictoris b at different epochs. Each row in the dataset corresponds to an observation and includes the following columns:
- `epoch`: the time $t$ of observation (in Modified Julian Date, MJD)
- `sep_obs`: measured separation [mas]
- `sep_err`: uncertainty in the measured separation [mas]
- `pa_obs`: measured position angle [degrees]
- `pa_err`: uncertainty in the measured position angle [degrees]

<div class="alert alert-success">
    
**Q6**. Use MCMC (via `emcee`) to sample from the posterior distribution of the orbital parameters.
- Assess and discuss autocorrelation, mixing, and convergence of the chains.
- Visualize the posterior distributions and compare them with the prior distributions.
    
</div>

<div class="alert alert-success">
    
**Q7**. Use variational inference (any variant) to approximate the posterior distribution of the orbital parameters.
- Assess the quality of the approximation.
- Visualize the variational posterior distributions and compare them with the prior distributions.
    
</div>

<div class="alert alert-success">

**Q8**. Compare and discuss MCMC and variational inference results.

</div>

# Critique: posterior predictive checks

<div class="alert alert-success">

**Q9**. Generate posterior predictive samples. Assess whether the observations fall within the posterior predictive intervals.

</div>

<div class="alert alert-success">
    
**Q10**. Create 
- a plot showing observed positions and posterior predicted orbits around the star (i.e., $\Delta \mathrm{RA}$ vs. $\Delta \mathrm{Dec}$).
- a plot showing observed angular displacement over time, along with posterior predictive intervals (i.e., $\Delta \mathrm{RA}$ vs. $t$ and $\Delta \mathrm{Dec}$ vs. $t$).


# Revise

<div class="alert alert-success">
    
**Q11**. Discuss potential improvements and extensions to this analysis.
    
</div>